# 11. Lab11-2 CNN MNIST: 99%

> Convolutional Neural Networks

### 2018.09.29 (토)

lab11-1 에서 진행한 CNN에서 CONV 와 POOL layer 를 하나씩 더 준다면.
![lec11_25](../../img/lec11_25.png)

In [1]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

/Users/Choi-seonyeol/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# hyper parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

## Input layer
X : 28 \* 28 \* 1 의  흑백 data <br>
Y : 0 ~ 9 사이의 숫자  class <br>
keep_prob : drop out 하지 않고, 사용할 node 의 비율

In [4]:
# input place holders
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])   # img 28x28x1 (black/white) -> -1 의 의미는, 알아서 결정하라는 뜻.
Y = tf.placeholder(tf.float32, [None, 10])

# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

## Conv layer 1

fillter : [3,3,1,32] -> 3\*3 짜리 흑백 image 로 각각 한장씩 32개 filter<br>
stride : [1,1,1,1] -> 한칸씩 이동<br>
padding : "SAME" -> 같은 size 가 적용되도록 zero-padding<br>
relu : ReLU 적용<br>
pool : kernel size : 1장 당 2*2 하나씩, strides : 2*2 로 두칸씩<br>
![lec11_26](../../img/lec11_26.png)

In [5]:
# L1 ImgIn shape=(?, 28, 28, 1)
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
#    Conv     -> (?, 28, 28, 32) -> (3*3*1), 32개의 fillter
#    Pool     -> (?, 14, 14, 32)
L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
# CONV
L1 = tf.nn.relu(L1)
# ReLU
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1], padding='SAME')
# POOL

'''
Tensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32) !
--> 결과 [14,14,32] -> 이것이 두번째 CONV 의 INPUT
'''

'\nTensor("Conv2D:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("Relu:0", shape=(?, 28, 28, 32), dtype=float32)\nTensor("MaxPool:0", shape=(?, 14, 14, 32), dtype=float32) !\n--> 결과 [14,14,32] -> 이것이 두번째 CONV 의 INPUT\n'

## Conv layer 2

fillter : [3,3,32,64] -> 3\*3 짜리 흑백 image 로 각각 한장씩 64개 filter
<br>
여기서, 앞의 Input 으로부터 depth 가 32이므로, 이를 받는다.
<br>
stride : [1,1,1,1] -> 한칸씩 이동<br>
padding : "SAME" -> 같은 size 가 적용되도록 zero-padding<br>
relu : ReLU 적용<br>
pool : kernel size : 1장 당 2*2 하나씩, strides : 2*2 로 두칸씩<br>
![lec11_27](../../img/lec11_27.png)

In [6]:
"""
INPUT : Tensor("dropout/mul:0", shape=(?, 14, 14, 32), dtype=float32)
"""
# L2 ImgIn shape=(?, 14, 14, 32)
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
#    Conv      ->(?, 14, 14, 64)
#    Pool      ->(?, 7, 7, 64)
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
L2 = tf.reshape(L2, [-1,7*7*64])
# 우리는 이제, L2를 FC layer의 Input으로 사용할 것인데, reshape을 하기 전까지는 3차원으로 구성되어있다.
# 이를 reshape 함수를 써서 1차원의 vector 로 변경해주어  Farward 학습을 진행한다.

'''
Tensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)
Tensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Reshape_1:0",shape = (?,3136),dtype = float32)
--> 결과 [1,3136] -> 이것이 FC layer의 INPUT
'''

'\nTensor("Conv2D_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("Relu_1:0", shape=(?, 14, 14, 64), dtype=float32)\nTensor("MaxPool_1:0", shape=(?, 7, 7, 64), dtype=float32)\nTensor("Reshape_1:0",shape = (?,3136),dtype = float32)\n--> 결과 [1,3136] -> 이것이 FC layer의 INPUT\n'

####  28\*28 짜리 이미지가 CONV 두번으로 1*3136 의 vector data 로 변환
__이제 이 데이터를 FC layer에 넣어 classification을 진행__

## Fully Connected(FC, Dense) Layer
Input : [7\*7\*64] -> [1\* 3136] 의  vector data<br>
Init method : Xavier_initializer<br>
Cost : softmax<br>
optimizer : Adam

![lec11_28](../../img/lec11_28.png)

In [7]:
"""
INPUT : Tensor("Reshape_1:0",shape = (?,3136),dtype = float)
"""

# Final FC 7x7x64 inputs -> 10outputs
W3 = tf.get_variable("W3",shape = [7*7*64,10],initializer = tf.contrib.layers.xavier_initializer_conv2d())
# 출력은 10개이기 때문에,shape의 모양에 주의
b = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b

# define cost/Loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = hypothesis, labels = Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

## Training and Evaluation

In [9]:
# initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

# train my model
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _, = sess.run([cost, optimizer], feed_dict = feed_dict)
        avg_cost += c / total_batch
    print("Epoch : ", "%04d" % (epoch + 1), "cost = ", "{:.9f}".format(avg_cost))
    
print("Learning Finished!")

Learning started. It takes sometime.
Epoch :  0001 cost =  0.341508756
Epoch :  0002 cost =  0.092470534
Epoch :  0003 cost =  0.070801435
Epoch :  0004 cost =  0.057510390
Epoch :  0005 cost =  0.049952429
Epoch :  0006 cost =  0.043398253
Epoch :  0007 cost =  0.038721363
Epoch :  0008 cost =  0.033889739
Epoch :  0009 cost =  0.029923751
Epoch :  0010 cost =  0.025600706
Epoch :  0011 cost =  0.023579466
Epoch :  0012 cost =  0.020005190
Epoch :  0013 cost =  0.019097268
Epoch :  0014 cost =  0.016151725
Epoch :  0015 cost =  0.014709784
Learning Finished!


In [ ]:
# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1),tf.argmax(Y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy : ",sess.run(accuracy, feed_dict = {X: mnist.test.images, Y : mnist.test.labels}))

## 여기서 더 DEEP  하게 갈 수도 있다.
CONV layer(Convolution + Pooling) 3개, FC layer 2개
![lec11_29](../../img/lec11_29.png)

--> __정확도가 매우 높아진다.__

이때, deep 할경우 Drop_out 을 꼭 해줘야 하는 것이 주의.<br>
또한, Dropout 시, 마지막 Test때는 dropout 하지 않고(keep_prob : 1) 로 해야하는 것또한 주의